## 🌐 Connect Colab to Google Drive

In [1]:
from google.colab import drive

drive.mount("/gdrive")
%cd /gdrive/My Drive
%cd [2024-2025] AN2DL Homework 2

Mounted at /gdrive
/gdrive/My Drive
/gdrive/My Drive/[2024-2025] AN2DL Homework 2


## ⚙️ Import Libraries

In [2]:
from datetime import datetime

import numpy as np
import pandas as pd
import logging
import random

import tensorflow as tf
from tensorflow import keras as tfk
tfk.config.enable_unsafe_deserialization()
from tensorflow.keras import layers as tfkl
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from scipy.stats import mode

import matplotlib.pyplot as plt
%matplotlib inline

seed = 29
np.random.seed(seed)
tf.random.set_seed(seed)

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

## ⏳ Load the Datasets

In [3]:
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")

# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

Training X shape: (2615, 64, 128)
Training y shape: (2615, 64, 128)
Test X shape: (10022, 64, 128)
Input shape: (64, 128, 1)
Number of classes: 5


## 🔍 Inspect the training dataset

In [ ]:
# Calculate prevalent labels
y_train_labels = mode(y_train, axis=(1, 2))[0].flatten()

print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train_labels: {y_train_labels.shape}")

# List all unique labels to check correctness
unique_labels = np.unique(y_train)
print(f"Unique classes: {unique_labels}")

# Plot images in batches
def plot_images(X, y, start_index=0, images_per_row=10, images_per_col=10):
    fig, axes = plt.subplots(images_per_col, images_per_row, figsize=(15, 15))
    for i in range(images_per_row * images_per_col):
        idx = start_index + i
        if idx >= len(X):
            break
        ax = axes[i // images_per_row, i % images_per_row]
        ax.imshow(X[idx], cmap="gray")
        ax.set_title(f"Class: {y[idx]}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

# Plot a sample image from each class
def plot_one_sample_per_class(X, y, y_mask, classes):
    for label in classes:
        for i in range(len(y_mask)):
            if label in np.unique(y_mask[i]):
                plt.figure()
                plt.imshow(X[i], cmap="gray")
                plt.title(f"Class: {label}")
                plt.axis("off")
                plt.show()
                break

plot_one_sample_per_class(X_train, y_train_labels, y_train, unique_labels)

# Plot all images
images_per_row = 10
images_per_col = 10
images_per_page = images_per_row * images_per_col
num_images = X_train.shape[0]

for start_idx in range(0, num_images, images_per_page):
    plot_images(X_train, y_train_labels, start_index=start_idx, images_per_row=images_per_row, images_per_col=images_per_col)

## ❌ Remove outliers from dataset

In [4]:
# Lists to contain filtered elements
X_train_filtered = []
y_train_filtered = []

for i in range(len(y_train)):
    label = y_train[i].argmax() if y_train.ndim > 1 else y_train[i]
    if label != 415:
        # Add to filtered dataset the non-alien images
        X_train_filtered.append(X_train[i])
        y_train_filtered.append(y_train[i])

# Convert lists to numpy arrays
X_train_filtered = np.array(X_train_filtered)
y_train_filtered = np.array(y_train_filtered)

print(f"Shape X_train_filtered: {X_train_filtered.shape}")
print(f"Shape y_train_filtered: {y_train_filtered.shape}")
print(f"Unique classes: {np.unique(y_train_filtered)}")

Shape X_train_filtered: (2505, 64, 128, 1)
Shape y_train_filtered: (2505, 64, 128)
Unique classes: [0. 1. 2. 3. 4.]


## 🔍 Inspect the filtered training dataset

In [ ]:
num_images_filtered = X_train_filtered.shape[0]
y_train_filtered_labels = mode(y_train_filtered, axis=(1, 2))[0].flatten()

# Plot the filtered dataset
for start_idx in range(0, num_images_filtered, images_per_page):
    plot_images(X_train_filtered, y_train_filtered_labels, start_index=start_idx, images_per_row=images_per_row, images_per_col=images_per_col)

## 🧮 Define network parameters

In [ ]:
# Set batch size for training
batch_size = 64

# Set learning rate for the optimiser
learning_rate = 1e-3

# Set early stopping patience threshold
patience = 10

# Set maximum number of training epochs
epochs = 100

## ✂ Split into Training and Validation Sets

In [ ]:
# Split the training dataset to get a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train_filtered,
    y_train_filtered,
    test_size=0.2,
    random_state=seed)

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weights = {i: weight for i, weight in enumerate(class_weights)}
class_weights = [class_weights[key] for key in sorted(class_weights.keys())]

# Print the shapes of the resulting sets
print('Training set shape:\t', X_train.shape, y_train.shape)
print('Validation set shape:\t', X_val.shape, y_val.shape)

# Convert in un tensore statico
class_weights = tf.constant(class_weights, dtype=tf.float32)
print(f"Class weights: {class_weights}")

Training set shape:	 (2004, 64, 128, 1) (2004, 64, 128)
Validation set shape:	 (501, 64, 128, 1) (501, 64, 128)
Class weights: [  0.8245642   0.5963      0.8436091   1.0896032 141.21344  ]


## 🔄 Preprocess Dataset

In [ ]:
def augment_data(image, label):
    # Geometric Transformations
    image = tf.image.random_flip_left_right(image)
    label = tf.image.random_flip_left_right(label)

    image = tf.image.random_flip_up_down(image)
    label = tf.image.random_flip_up_down(label)

    # Chromatic Transformations
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)

    return image, label

def preprocess_image(image):
    image = tf.expand_dims(image, axis=-1) if len(image.shape) == 2 else image
    image = tf.cast(image, tf.float32)
    return image

def preprocess_label(label):
    label = tf.expand_dims(label, axis=-1) if len(label.shape) == 2 else label
    label = tf.cast(label, tf.int32)
    return label

def preprocess_data(image, label):
    image = preprocess_image(image)
    label = preprocess_label(label)
    return image, label

In [ ]:
# Original dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Augmented dataset
augmented_dataset = train_dataset.map(lambda x, y: augment_data(x, y), num_parallel_calls=tf.data.AUTOTUNE)

# Combined dataset, having both augmented and original dataset
combined_dataset = train_dataset.concatenate(augmented_dataset)
combined_dataset = combined_dataset.shuffle(buffer_size=len(X_train))
combined_dataset = combined_dataset.prefetch(tf.data.AUTOTUNE)

# Validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

def pixel_distribution(dataset, num_classes):
    class_counts = {i: 0 for i in range(num_classes)}
    for _, label in dataset.unbatch():
        unique, counts = np.unique(label.numpy(), return_counts=True)
        for u, c in zip(unique, counts):
            class_counts[int(u)] += c
    return class_counts

print("Combined Dataset pixel distribution:\t", pixel_distribution(combined_dataset, num_classes))

Combined Dataset pixel distribution:	 {0: 7963852, 1: 11012422, 2: 7784064, 3: 6026696, 4: 46502}


## 🔨 Build the model

In [ ]:
def se_block(input_tensor, reduction_ratio=16):
    filters = input_tensor.shape[-1]
    se = tfkl.GlobalAveragePooling2D()(input_tensor)
    se = tfkl.Dense(filters // reduction_ratio, activation='relu', kernel_initializer='he_normal')(se)
    se = tfkl.Dense(filters, activation='sigmoid', kernel_initializer='he_normal')(se)
    se = tfkl.Reshape((1, 1, filters))(se)
    return tfkl.Multiply()([input_tensor, se])

class AdaptShapeLayer(tfkl.Layer):
    def __init__(self, **kwargs):
        super(AdaptShapeLayer, self).__init__(**kwargs)

    def call(self, tensor, target_tensor):
        target_shape = tf.shape(target_tensor)[1:3]
        tensor = tf.image.resize(tensor, target_shape, method='bilinear')
        return tensor

def adapt_channels(tensor, target_channels):
    conv = tfkl.Conv2D(target_channels, kernel_size=1, padding='same')
    return conv(tensor)

def bottleneck_layer(input_tensor, filters, reduction_ratio=4):
    adapt_shape = AdaptShapeLayer()
    residual = adapt_channels(input_tensor, filters)

    # Bottleneck structure
    reduced_filters = filters // reduction_ratio
    x = tfkl.Conv2D(reduced_filters, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(input_tensor)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.SeparableConv2D(filters, kernel_size=3, padding='same', activation='relu')(x)
    x = tfkl.BatchNormalization()(x)
    x = se_block(x)
    x = tfkl.Dropout(0.3)(x)
    residual = adapt_shape(residual, x)
    x = tfkl.Add()([x, residual])
    return x

def unet_block(input_tensor, filters, kernel_size=3, stack=4, name=''):
    x = input_tensor
    adapt_shape = AdaptShapeLayer()

    # Save for residual connection
    residual = adapt_channels(input_tensor, filters)

    for i in range(stack):
        x = tfkl.Conv2D(filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(x)
        x = tfkl.BatchNormalization()(x)
        x = se_block(x)
        x = tfkl.Activation(tf.nn.leaky_relu)(x)
        x = tfkl.Conv2D(filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(x)
        x = tfkl.BatchNormalization()(x)
        x = se_block(x)
        x = tfkl.Activation(tf.nn.leaky_relu)(x)
        x = tfkl.SpatialDropout2D(0.2)(x)

    # Adapt shapes for residual connection
    residual = adapt_shape(residual, x)
    x = tfkl.Add()([x, residual])
    return x

In [ ]:
def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes, axis=-1)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)
    return 1 - dice

def focal_loss(gamma=2., alpha=0.25):
    gamma = tf.constant(gamma, dtype=tf.float32)
    alpha = tf.constant(alpha, dtype=tf.float32)

    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=y_pred.shape[-1])
        if len(y_true_one_hot.shape) > len(y_pred.shape):
            y_true_one_hot = tf.squeeze(y_true_one_hot, axis=-2)
        cross_entropy = -y_true_one_hot * tf.keras.backend.log(y_pred)
        weight = alpha * tf.math.pow((1 - y_pred), gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))

    return focal_loss_fixed

def combined_loss(y_true, y_pred, dice_weight=0.5, focal_weight=0.5):
    dice = dice_loss(y_true, y_pred, smooth=1e-6)
    focal = focal_loss(gamma=2.0, alpha=0.25)(y_true, y_pred)
    return dice_weight * dice + focal_weight * focal

In [ ]:
inputs = tfkl.Input(shape=input_shape)

# Downsampling
down_block_1 = unet_block(inputs, 64)
d1 = tfkl.MaxPooling2D()(down_block_1)

down_block_2 = unet_block(d1, 128)
d2 = tfkl.MaxPooling2D()(down_block_2)

down_block_3 = unet_block(d2, 256)
d3 = tfkl.MaxPooling2D()(down_block_3)

down_block_4 = unet_block(d3, 512)
d4 = tfkl.MaxPooling2D()(down_block_4)

# Bottleneck
bottleneck = bottleneck_layer(d4, 1024)

# Upsampling
u1 = tfkl.Conv2DTranspose(512, kernel_size=2, strides=2, padding='same')(bottleneck)
u1 = tfkl.Concatenate()([u1, down_block_4])
u1 = unet_block(u1, 512)

u2 = tfkl.Conv2DTranspose(256, kernel_size=2, strides=2, padding='same')(u1)
u2 = tfkl.Concatenate()([u2, down_block_3])
u2 = unet_block(u2, 256)

u3 = tfkl.Conv2DTranspose(128, kernel_size=2, strides=2, padding='same')(u2)
u3 = tfkl.Concatenate()([u3, down_block_2])
u3 = unet_block(u3, 128)

u4 = tfkl.Conv2DTranspose(64, kernel_size=2, strides=2, padding='same')(u3)
u4 = tfkl.Concatenate()([u4, down_block_1])
u4 = unet_block(u4, 64)

# Output
outputs = tfkl.Conv2D(num_classes, kernel_size=1, activation='softmax')(u4)

model = tfk.Model(inputs=inputs, outputs=outputs)

# Define the MeanIoU ignoring the background class
mean_iou = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0, sparse_y_pred=False, name='mean_iou')
optimizer = tfk.optimizers.AdamW(learning_rate=learning_rate, weight_decay=1e-5)
loss = combined_loss

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=[mean_iou])

## 🛠️ Train and Save the Model

In [ ]:
# Create an EarlyStopping callback
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=patience,
    restore_best_weights=True
)

# Create a LearningRate Scheduler
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5
)

# Store the callbacks in a list
callbacks = [early_stopping, lr_scheduler]

In [ ]:
history = model.fit(
    combined_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    batch_size=batch_size,
    callbacks=callbacks
).history

# Calculate and print the final validation accuracy
final_val_meanIoU = round(max(history['val_mean_iou'])* 100, 2)
print(f'Final validation Mean Intersection Over Union: {final_val_meanIoU}%')

Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 373s 3s/step - loss: 0.5272 - mean_iou: 0.1076 - val_loss: 0.4752 - val_mean_iou: 0.0937 - learning_rate: 0.0010
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 213ms/step - loss: 0.4494 - mean_iou: 0.1567 - val_loss: 0.5381 - val_mean_iou: 0.1319 - learning_rate: 0.0010
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 218ms/step - loss: 0.4226 - mean_iou: 0.2026 - val_loss: 0.4353 - val_mean_iou: 0.1420 - learning_rate: 0.0010
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 13s 212ms/step - loss: 0.4009 - mean_iou: 0.2259 - val_loss: 0.4993 - val_mean_iou: 0.1169 - learning_rate: 0.0010
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 214ms/step - loss: 0.3865 - mean_iou: 0.2354 - val_loss: 0.5124 - val_mean_iou: 0.1059 - learning_rate: 0.0010
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 213ms/step - loss: 0.3733 - mean_iou: 0.2587 - val_loss: 0.4795 - val_mean_iou: 0.1350 - learning_rate: 0.0010
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 216ms/step - loss: 0.3715 - mean_io

In [ ]:
timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)
del model

## 📊 Test the model



In [ ]:
model = tfk.models.load_model(model_filename, custom_objects={
        'dice_loss': dice_loss,
        'focal_loss': focal_loss,
        'combined_loss': combined_loss,
        'unet_block': unet_block,
        'bottleneck_layer': bottleneck_layer,
        'se_block': se_block,
        'AdaptShapeLayer': AdaptShapeLayer,
        'adapt_channels': adapt_channels
    }
)

preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 24s 55ms/step
Predictions shape: (10022, 64, 128)


## 💾 Save the predictions

In [ ]:
def y_to_df(y) -> pd.DataFrame:
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [ ]:
# Create the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)